# Libraries

In [1]:
import tensorflow as tf
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import random
from tqdm import tqdm

In [2]:
# If model already trained, then 0; otherwise, 1.
train_model = 0

# Data

## Data preparation

In [3]:
if train_model:
    # Paths to images
    image_path_arnau = "data/Arnau"
    image_path_ashley = "data/Ashley"
    image_path_megha = "data/Megha"

    # Prepare list with image paths
    file_list_arnau = os.listdir(image_path_arnau)
    file_list_arnau = [ f for f in file_list_arnau if f.lower().endswith('.jpeg') or f.lower().endswith('.jpg')]

    file_list_ashley = os.listdir(image_path_ashley)
    file_list_ashley = [ f for f in file_list_ashley if f.lower().endswith('.jpeg') or f.lower().endswith('.jpg')]

    file_list_megha = os.listdir(image_path_megha)
    file_list_megha = [ f for f in file_list_megha if f.lower().endswith('.jpeg') or f.lower().endswith('.jpg')]

    # Join lists and create labels
    file_list = file_list_arnau + file_list_ashley + file_list_megha
    labels = np.array([0] * len(file_list_arnau) + [1] * len(file_list_ashley) + [2] * len(file_list_megha))

    # Check that file_list and labels have the same shape
    print(len(file_list) == len(labels))

    # Print length for every group of faces
    unique_counts = np.unique(labels,return_counts=True)
    print(len(file_list_arnau),len(file_list_ashley),len(file_list_megha),unique_counts[1])

## Data import

### Functions for face detection

In [4]:
# Find center of boundary box and return it.
def findCenterBbox(bbox):
    x_min, y_min, w, h = bbox
    x_max = x_min + w
    y_max = y_min + h

    center = [int((x_max + x_min)//2), int((y_max + y_min)//2)] 

    return center

In [5]:
# Find face using the OpenCV library and return boundary box.
def findFaces(img):
    imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    faceDetection = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    results = faceDetection.detectMultiScale(imgGray, 1.1, 10, minSize=[int(0.25*imgGray.shape[0]),int(0.25*imgGray.shape[1])])

    bboxs = []
    
    if len(results):
        counter = 1
        for bbox in results:
            center_bbox = findCenterBbox(bbox)
            bboxs.append([counter, bbox, center_bbox])
            counter = counter + 1

    return bboxs

In [6]:
# Find the dimensions of a square box based on the center and the dimensions.
def makeSquareCrop(x_min, y_min, w, h):
    centerX = x_min + round(w/2)
    centerY = y_min + round(h/2)
    dim = max([w,h])

    x_min = centerX - round(dim/2)
    y_min = centerY - round(dim/2)

    w = dim
    h = dim

    return x_min, y_min, w, h

In [7]:
# Check if the boundaries are valid.
def boundariesValidation(x, y):
    x = max([x, 0])
    y = max([y, 0])

    return x, y

In [8]:
# Crop image and return it.
def crop(img, bbox):
    x_min, y_min, w, h = bbox

    x_min, y_min, w, h = makeSquareCrop(x_min, y_min, w, h)        

    x_min, y_min = boundariesValidation(x_min, y_min)

    return img[y_min:y_min+h, x_min:x_min+w]

### Data retrieval

In [9]:
input_size = [64,64] # Input size of image
if train_model:

    # Initialize img_feature to store the images
    img_features = np.empty([1,input_size[0], input_size[1],3])
    idx_keep = []

    # For loop to iterate through all files
    for i in tqdm(range(1,len(file_list))):

        f = file_list[i] # Current file

        # Path based on label
        if labels[i] == 2:
            path = image_path_megha + '/' + f
        if labels[i] == 1:
            path = image_path_ashley + '/' + f
        elif labels[i] == 0:
            path = image_path_arnau + '/' + f

        # Skip iteration if image cannot be loaded
        img = cv2.imread(path)
        if img is None:
            continue

        bboxs = findFaces(img) # Find face

        # If no faces have been found, skip face
        if(len(bboxs) == 0):
            continue     

        # Crop faces in square dimensions
        img_face = crop(img, bboxs[0][1])

        # Resize image into input_size and convert BGR to RGB
        img_face_ds = cv2.resize(img_face, (input_size[0],input_size[1]))
        img_face_ds = cv2.cvtColor(img_face_ds, cv2.COLOR_BGR2RGB)

        # Reshape image and append it to img_features
        features = np.reshape(img_face_ds,[1,input_size[0],input_size[1],3])
        img_features = np.append(img_features,features,axis=0)

        idx_keep.append(i) # Indices of images with detected faces

    # Delete positions where no faces have been detected
    img_features = np.delete(img_features,obj=0,axis=0)
    labels = labels[idx_keep]

    N_samples = img_features.shape[0] # Compute number of samples
    
    # Print number of detected faces for every class
    unique_counts = np.unique(labels,return_counts=True)
    print(unique_counts[0],unique_counts[1],len(labels))

# Deep learning model

## Convolutional Neural Network (CNN) architecture

In [10]:
if train_model:
    model = tf.keras.Sequential(
        [
        tf.keras.layers.Rescaling(1./255, input_shape=((input_size[0], input_size[1], 3))),
        tf.keras.layers.Conv2D(input_size[0], (3, 3), activation='relu', input_shape=(input_size[0], input_size[1], 3)),
        tf.keras.layers.MaxPooling2D((2, 2)),

        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),

        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(len(np.unique(labels)))
        ]
    )

## Model fitting

In [11]:
if train_model:
    # Define indices of training (70%) and testing (30%)
    train_indices = random.sample(range(0, N_samples), round(N_samples*0.7)) #int(np.floor(img_features.shape[0]*0.7))
    test_indices = list(set(list(range(0,N_samples))).difference(set(train_indices)))

    # Split images into training and testing
    train = img_features[train_indices,:,:,:]
    train_labels = labels[train_indices]

    test = img_features[test_indices,:,:,:]
    test_labels = labels[test_indices]

    # Train CNN model
    model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

    history = model.fit(train, train_labels, epochs=100, validation_data=(test, test_labels))

    plt.plot(history.history['accuracy'], label='accuracy')
    plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.ylim([0, 1])
    plt.legend(loc='lower right')

    test_loss, test_acc = model.evaluate(test,  test_labels, verbose=2)

In [12]:
if train_model:
    # Save model
    model.save('data/my_model')

# Testing the model

In [ ]:
model = tf.keras.models.load_model('data/my_model') # Import model
cap = cv2.VideoCapture(0) # Load webcam
class_names = ['Arnau', 'Ashley', 'Megha'] # Classes names

# Check if the webcam is opened correctly
if not cap.isOpened():
    raise IOError("Cannot open webcam")

# Real-time face recognition
while True:
    ret, frame = cap.read()
    frame = cv2.resize(frame, None, fx=0.5, fy=0.5,
                       interpolation=cv2.INTER_AREA)

    bboxs = findFaces(frame)

    c = cv2.waitKey(1)

    if c == 27:  # Escape
        cap.release()
        break

    for i in range(0, len(bboxs)):
        bbox = bboxs[i][1]
        img_face = crop(frame, bbox)
        img_face_ds = cv2.resize(img_face, (input_size[0], input_size[1]))
        img_face_ds = cv2.cvtColor(img_face_ds, cv2.COLOR_BGR2RGB)

        predictions = model.predict(
            np.reshape(img_face_ds, [1, img_face_ds.shape[0], img_face_ds.shape[1], 3]))
        score = tf.nn.softmax(predictions[0])
        pred_class = class_names[np.argmax(score)]
        pred_score = 100 * np.max(score)
        cv2.rectangle(
            frame, (bbox[0], bbox[1]), (bbox[0]+bbox[2], bbox[1]+bbox[3]), (255, 0, 0), 2)
        cv2.putText(
            frame, pred_class + ' | ' + str(int(pred_score)) + '%', org=(bbox[0]+bbox[2], bbox[1]), fontFace=cv2.FONT_HERSHEY_PLAIN, fontScale=1, color=(255, 0, 0))

    cv2.imshow('Input', frame)

cv2.destroyAllWindows()
_ = cv2.waitKey(1) # Bug for MacOS (window does not close)

Metal device set to: Apple M1


2022-03-11 15:08:44.692776: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-03-11 15:08:44.698644: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-03-11 15:08:47.539042: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-03-11 15:08:47.615825: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
